In [2]:
import requests
from bs4 import BeautifulSoup
import time
import json

In [3]:
def get_flight() -> dict:
    ignore = ["full_count", "version", "stats"]
    url = "https://data-live.flightradar24.com/zones/fcgi/feed.js?bounds=59.09,52.64,-58.77,-47.71&faa=1&mlat=1&flarm=1&adsb=1&gnd=1&air=1&vehicles=1&estimated=1&maxage=7200&gliders=1&stats=1"

    # Request with fake header, otherwise you will get an 403 HTTP error
    r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})

    # Parse the JSON
    data = r.json()
    [ data.pop(x) for x in ignore ]
    return data

In [4]:
def parser(data: dict) -> dict:
    clean_data = {}
    
    # Parameters returned of request
    fields = ["Model-S", "Latitude", "Longitude", 3, 4, 5, 6, "Feeder-Station-Code", "Aircraft-Model", "Aircraft-Registration", "Timestamp", "From", "To", "Flight-Code", 14, 15, "Airline-Flight-Code", 17, "Airline"]

    # Build dict to write in JSON
    for key in data.keys():
        clean_data[key] = {}
        for field, value in zip(fields, data[key]):
            clean_data[key].update({field:value})

    return clean_data

In [5]:
x = 0

with open("ads-b_data.json", "w+") as ads_b_data:
    while x < 10:
        try:
            json.dump(parser(get_flight()), ads_b_data, indent=2, separators=(",", ":"))
            time.sleep(8)
        except KeyboardInterrupt:
            exit(0)
        x += 1